# Behavioral Analysis of Ship–Ship Interactions from AIS Data

## Purpose of this Notebook

This notebook performs a behavioral analysis of ship–ship interactions using
AIS-derived kinematic data. The goal is to characterize how vessels behave
(speed and course changes) when operating in close proximity to another ship,
especially in situations associated with potential collision risk.

The analysis focuses on **interpretable behavioral summaries** rather than
trajectory reconstruction or prediction. Each interaction is reduced to a
compact set of features describing how both ships responded during the interaction.

This notebook is designed to support:
- Exploratory behavioral analysis
- Statistical comparison of interaction types
- Future development of a behavior-based classifier (handled separately)

---

## Input Data

The input file `classified_ais_dcpa_tcpa.csv` contains time-aligned AIS data
for pairs of ships. Each row represents a single timestamp and includes:

- Position (latitude, longitude) for both ships
- Speed over ground and course over ground
- Distance between ships
- Risk indicators: DCPA (Distance at Closest Point of Approach) and TCPA
- Region classification (harbor / open sea)

The data has already been filtered to represent ship pairs that come into
meaningful proximity.

---

## Canonical Pair Definition (Important)

AIS pair data may appear in both orientations:
- `(mmsi_1=A, mmsi_2=B)` and later `(mmsi_1=B, mmsi_2=A)`

To ensure these refer to the same ship pair, this notebook creates a
**canonical pair identifier**:

- `ship_low  = min(mmsi_1, mmsi_2)`
- `ship_high = max(mmsi_1, mmsi_2)`

All interaction grouping and indexing is performed using `(ship_low, ship_high)`
so that both orientations are treated as a single ship pair.

---

## Interaction Definition

Ship–ship interactions are defined by grouping consecutive AIS observations
for the same canonical ship pair (`ship_low`, `ship_high`).

A new interaction is started when the time gap between observations exceeds
60 minutes. This permissive threshold reflects realistic maritime behavior,
where ships may maneuver slowly and interactions can unfold over extended periods.

Each interaction represents a *single navigational situation* between two ships.

---

## Behavioral Analysis Approach

For each interaction, the following aspects of ship behavior are summarized:

- Speed behavior (net speed change, peak acceleration)
- Course behavior (total course change, peak turn rate)
- Risk context (minimum DCPA and associated TCPA)
- Qualitative maneuver interpretation:
  - Whether a ship maneuvered
  - Whether the maneuver was speed-dominant or course-dominant

The result is a compact dataset with one row per interaction, suitable for analysis
and for later use as input features to a simple classifier.


In [1]:
import pandas as pd
import numpy as np

# ============================
# CONFIG (simple & permissive)
# ============================
INPUT_FILE = "classified_ais_dcpa_tcpa.csv"
MAX_TIME_GAP_SEC = 3600   # 60 minutes → ships can maneuver slowly

KNOTS_TO_MPS = 0.514444

# ============================
# Helper functions
# ============================
def angle_diff_deg(a, b):
    """Smallest signed angle difference a-b in degrees [-180, 180]."""
    return ((a - b + 180) % 360) - 180

# ============================
# Load
# ============================
df = pd.read_csv(INPUT_FILE, parse_dates=["time_window"])

# ============================
# CANONICAL PAIR (FIX)
# ship_low/ship_high makes (A,B) and (B,A) the SAME pair
# ============================
df["ship_low"]  = df[["mmsi_1", "mmsi_2"]].min(axis=1)
df["ship_high"] = df[["mmsi_1", "mmsi_2"]].max(axis=1)


In [2]:
# ============================
# CANONICAL ORIENTATION + DEDUP (IMPORTANT)
# ============================
# Why:
# The source pair dataset can contain symmetric duplicates:
# (A,B) and (B,A) for the same timestamp. Even after ship_low/ship_high,
# these duplicates can "flip" which vessel appears in the *_1 vs *_2 columns.
# That would create artificial jumps in speed/course when we compute diffs,
# inflating turn/acceleration features.
#
# Fix:
# 1) Force a consistent orientation: mmsi_1 == ship_low and mmsi_2 == ship_high
#    by swapping the *_1 and *_2 columns when needed.
# 2) Drop duplicates per (ship_low, ship_high, time_window).

# Identify rows where the order is flipped (mmsi_1 is actually the high ship)
flip_mask = df["mmsi_1"] != df["ship_low"]

# Columns that must be swapped to keep ship_1/ship_2 consistent
swap_cols = ["mmsi", "lat", "lon", "speed", "course"]

for c in swap_cols:
    col1 = f"{c}_1"
    col2 = f"{c}_2"
    tmp = df.loc[flip_mask, col1].copy()
    df.loc[flip_mask, col1] = df.loc[flip_mask, col2].values
    df.loc[flip_mask, col2] = tmp.values

# After swapping, mmsi_1 should always equal ship_low (sanity check)
if not (df["mmsi_1"] == df["ship_low"]).all():
    bad = df[df["mmsi_1"] != df["ship_low"]].head(5)
    raise ValueError(
        "Canonical orientation failed: mmsi_1 != ship_low still exists. "
        "Example rows:\n" + bad.to_string(index=False)
    )

# Drop duplicate timestamps for the same canonical pair
before = len(df)
df = df.drop_duplicates(subset=["ship_low", "ship_high", "time_window"]).reset_index(drop=True)
after = len(df)

print(f"✅ Canonicalized + deduplicated: removed {before - after} duplicate rows "
      f"({before} → {after})")


✅ Canonicalized + deduplicated: removed 160090 duplicate rows (677044 → 516954)


In [3]:
# ============================
# Sort (use canonical pair)
# ============================
df = df.sort_values(["ship_low", "ship_high", "time_window"]).reset_index(drop=True)

# ============================
# Define interactions (simple)
# ============================
df["time_diff_s"] = (
    df.groupby(["ship_low", "ship_high"])["time_window"]
      .diff()
      .dt.total_seconds()
)

df["new_interaction"] = df["time_diff_s"].isna() | (df["time_diff_s"] > MAX_TIME_GAP_SEC)

df["interaction_id"] = (
    df.groupby(["ship_low", "ship_high"])["new_interaction"]
      .cumsum()
      .astype(int) - 1
)

# ============================
# Build behavioral summary
# ============================
def summarize_behavior(g: pd.DataFrame) -> pd.Series:
    g = g.sort_values("time_window")

    n = len(g)
    start_time = g["time_window"].iloc[0]
    end_time = g["time_window"].iloc[-1]
    duration_min = (end_time - start_time).total_seconds() / 60.0

    # --- Risk context ---
    min_dcpa_m = g["DCPA_m"].min()
    idx_min = g["DCPA_m"].idxmin()
    tcpa_s = g.loc[idx_min, "TCPA_s"] if pd.notna(idx_min) else np.nan

    # --- Speed behavior ---
    net_speed_change_1 = g["speed_1"].iloc[-1] - g["speed_1"].iloc[0]
    net_speed_change_2 = g["speed_2"].iloc[-1] - g["speed_2"].iloc[0]

    dt = g["time_window"].diff().dt.total_seconds()
    dt_safe = dt.where(dt > 0)

    accel_1 = g["speed_1"].diff() * KNOTS_TO_MPS / dt_safe
    accel_2 = g["speed_2"].diff() * KNOTS_TO_MPS / dt_safe

    max_accel_1 = accel_1.abs().max()
    max_accel_2 = accel_2.abs().max()

    # --- Course behavior ---
    turn_1 = angle_diff_deg(g["course_1"], g["course_1"].shift(1))
    turn_2 = angle_diff_deg(g["course_2"], g["course_2"].shift(1))

    total_turn_1 = turn_1.abs().sum()
    total_turn_2 = turn_2.abs().sum()

    turn_rate_1 = turn_1 / dt_safe
    turn_rate_2 = turn_2 / dt_safe

    max_turn_rate_1 = turn_rate_1.abs().max()
    max_turn_rate_2 = turn_rate_2.abs().max()

    maneuvered_1 = total_turn_1 > 5 or abs(net_speed_change_1) > 0.5
    maneuvered_2 = total_turn_2 > 5 or abs(net_speed_change_2) > 0.5

    dominance_1 = "course" if total_turn_1 > abs(net_speed_change_1) * 10 else "speed"
    dominance_2 = "course" if total_turn_2 > abs(net_speed_change_2) * 10 else "speed"

    return pd.Series({
        # Keep canonical pair in summary for reliable linking

        "start_time": start_time,
        "end_time": end_time,
        "duration_min": duration_min,
        "points_count": n,

        "min_dcpa_m": min_dcpa_m,
        "tcpa_s": tcpa_s,

        "net_speed_change_1_kn": net_speed_change_1,
        "net_speed_change_2_kn": net_speed_change_2,
        "max_accel_1_mps2": max_accel_1,
        "max_accel_2_mps2": max_accel_2,

        "total_turn_1_deg": total_turn_1,
        "total_turn_2_deg": total_turn_2,
        "max_turn_rate_1_deg_s": max_turn_rate_1,
        "max_turn_rate_2_deg_s": max_turn_rate_2,

        "maneuvered_1": maneuvered_1,
        "maneuvered_2": maneuvered_2,
        "dominance_1": dominance_1,
        "dominance_2": dominance_2,

        "region_type": g["region_type"].mode().iat[0] if not g["region_type"].mode().empty else None
    })

behavior_summary = (
    df.groupby(["ship_low", "ship_high", "interaction_id"], sort=False)
      .apply(summarize_behavior)
      .reset_index()
)


behavior_summary["mmsi_1_example"] = (
    df.groupby(["ship_low","ship_high","interaction_id"])["mmsi_1"].first().values
)
behavior_summary["mmsi_2_example"] = (
    df.groupby(["ship_low","ship_high","interaction_id"])["mmsi_2"].first().values
)

# ============================
# Save result
# ============================
behavior_summary.to_csv("behavior_summary.csv", index=False)
print("✅ Saved behavior_summary.csv")


✅ Saved behavior_summary.csv


In [4]:
behavior_summary.head(20)

,ship_low,ship_high,interaction_id,start_time,end_time,duration_min,points_count,min_dcpa_m,tcpa_s,net_speed_change_1_kn,...,total_turn_2_deg,max_turn_rate_1_deg_s,max_turn_rate_2_deg_s,maneuvered_1,maneuvered_2,dominance_1,dominance_2,region_type,mmsi_1_example,mmsi_2_example
0,923166,232004686,0,2015-12-01 17:43:00,2015-12-01 17:43:00,0.0,1,492.766201,68.166279,0.0,...,0.0,NaN,NaN,False,False,speed,speed,open_sea,923166,232004686
1,205067000,248843000,0,2015-10-23 01:43:00,2015-10-23 01:48:00,5.0,2,863.891201,0.000000,0.0,...,0.0,0.000333,0.000000,False,False,course,speed,open_sea,205067000,248843000
2,205067000,248843000,1,2015-10-23 03:38:00,2015-10-23 05:48:00,130.0,5,1346.859584,0.000000,1.2,...,9.0,0.001000,0.008333,True,True,speed,speed,open_sea,205067000,248843000
3,205067000,248843000,2,2015-10-23 07:33:00,2015-10-23 07:43:00,10.0,2,95.337561,4166.337347,-0.1,...,1.8,0.000833,0.003000,False,False,speed,course,open_sea,205067000,248843000
4,205204000,211211220,0,2015-11-19 03:48:00,2015-11-19 06:36:00,168.0,19,2.128205,3793.636349,-6.0,...,211.2,0.055556,0.206667,True,True,course,course,open_sea,205204000,211211220
5,205204000,211211220,1,2015-11-23 10:15:00,2015-11-23 10:30:00,15.0,14,753.050060,0.000000,6.8,...,197.5,0.600000,0.806667,True,True,course,course,harbor,205204000,211211220
6,205204000,227005550,0,2015-10-02 06:22:00,2015-10-02 07:31:00,69.0,21,7.702577,31.582827,-8.7,...,1079.8,0.800000,2.953333,True,True,course,course,harbor,205204000,227005550
7,205204000,227008170,0,2015-10-02 07:32:00,2015-10-02 07:35:00,3.0,4,909.941919,76.710659,0.0,...,15.0,0.116667,0.216667,True,True,course,course,harbor,205204000,227008170
8,205204000,227406000,0,2015-10-07 09:14:00,2015-10-07 09:14:00,0.0,1,1521.773571,96.482545,0.0,...,0.0,NaN,NaN,False,False,speed,speed,open_sea,205204000,227406000
9,205204000,227574020,0,2015-10-02 07:34:00,2015-10-02 07:35:00,1.0,2,1814.323109,16.243705,-0.2,...,8.8,0.083333,0.146667,False,True,course,course,harbor,205204000,227574020


In [6]:
behavior_summary.describe()

,ship_low,ship_high,interaction_id,duration_min,points_count,min_dcpa_m,tcpa_s,net_speed_change_1_kn,net_speed_change_2_kn,max_accel_1_mps2,max_accel_2_mps2,total_turn_1_deg,total_turn_2_deg,max_turn_rate_1_deg_s,max_turn_rate_2_deg_s,mmsi_1_example,mmsi_2_example
count,2.694600e+04,2.694600e+04,26946.000000,26946.000000,26946.000000,26943.000000,26943.000000,26946.000000,26946.000000,22731.000000,22731.000000,26946.000000,26946.000000,22731.000000,22731.000000,2.694600e+04,2.694600e+04
mean,2.276525e+08,2.377944e+08,16.252357,43.530580,19.184814,510.480615,405.060725,0.057600,-0.228386,0.024422,0.021224,515.283693,486.180505,1.055766,0.942913,2.276525e+08,2.377944e+08
std,6.036746e+06,6.018122e+07,22.278563,64.754765,32.373494,547.231390,3488.388145,5.489664,5.268446,0.048720,0.055282,1013.437149,1005.004229,1.005304,0.979862,6.036746e+06,6.018122e+07
min,9.231660e+05,2.112112e+08,0.000000,0.000000,1.000000,0.000988,0.000000,-101.600000,-101.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.231660e+05,2.112112e+08
25%,2.273151e+08,2.276328e+08,2.000000,3.000000,3.000000,46.723976,4.714488,-1.200000,-1.100000,0.004287,0.002572,12.600000,5.900000,0.194028,0.141667,2.273151e+08,2.276328e+08
50%,2.275805e+08,2.276865e+08,9.000000,12.000000,7.000000,268.113327,112.349915,0.000000,0.000000,0.014576,0.010289,122.050000,95.700000,0.645000,0.536667,2.275805e+08,2.276865e+08
75%,2.276352e+08,2.281867e+08,22.000000,60.000000,20.000000,911.503003,314.509538,1.000000,0.600000,0.036011,0.029152,492.300000,416.675000,1.850000,1.500833,2.276352e+08,2.281867e+08
max,6.360150e+08,1.000000e+09,153.000000,1351.000000,357.000000,1846.752216,508829.434399,101.500000,101.600000,0.871983,0.871983,13718.100000,21656.900000,3.000000,3.000000,6.360150e+08,1.000000e+09


# Output Description: behavior_summary.csv

Each row in `behavior_summary.csv` represents one ship–ship interaction.
The columns are defined as follows:

---

## Identification Columns

- **ship_low**  
  The smaller MMSI of the ship pair (canonical ordering).

- **ship_high**  
  The larger MMSI of the ship pair (canonical ordering).

- **interaction_id**  
  Sequential identifier for interactions within the same canonical pair
  `(ship_low, ship_high)`. IDs start at 0 for each canonical pair.

---

## Temporal Characteristics

- **start_time**  
  Timestamp of the first AIS observation in the interaction.

- **end_time**  
  Timestamp of the last AIS observation in the interaction.

- **duration_min**  
  Total duration of the interaction in minutes.

- **points_count**  
  Number of AIS observations contained in the interaction.

---

## Risk Context Metrics

- **min_dcpa_m**  
  Minimum Distance at Closest Point of Approach (DCPA) observed during the
  interaction, in meters.

- **tcpa_s**  
  Time to Closest Point of Approach (TCPA) associated with the minimum DCPA,
  in seconds.

---

## Speed Behavior Metrics

> **Important note on ship identity:**  
> Behavioral metrics with suffixes `_1` and `_2` are computed from the
> original AIS columns (`speed_1`, `speed_2`, etc.).  
> After canonicalization and deduplication, these streams are **internally
> consistent within each interaction**, but their correspondence to the
> physical vessels should be interpreted using the reference columns
> `mmsi_1_example` and `mmsi_2_example`.

- **net_speed_change_1_kn**  
  Net change in speed (knots) of the vessel represented by the `_1` data stream
  from the start to the end of the interaction.

- **net_speed_change_2_kn**  
  Net change in speed (knots) of the vessel represented by the `_2` data stream.

- **max_accel_1_mps2**  
  Maximum absolute acceleration of the `_1` vessel during the interaction,
  in m/s².

- **max_accel_2_mps2**  
  Maximum absolute acceleration of the `_2` vessel during the interaction,
  in m/s².

---

## Course Behavior Metrics

- **total_turn_1_deg**  
  Total accumulated absolute course change of the `_1` vessel
  during the interaction, in degrees.

- **total_turn_2_deg**  
  Total accumulated absolute course change of the `_2` vessel
  during the interaction, in degrees.

- **max_turn_rate_1_deg_s**  
  Maximum absolute course change rate of the `_1` vessel, in deg/s.

- **max_turn_rate_2_deg_s**  
  Maximum absolute course change rate of the `_2` vessel, in deg/s.

---

## Behavioral Interpretation

- **maneuvered_1**  
  Boolean flag indicating whether the `_1` vessel performed a meaningful
  maneuver (speed or course change) during the interaction.

- **maneuvered_2**  
  Boolean flag indicating whether the `_2` vessel performed a meaningful
  maneuver.

- **dominance_1**  
  Dominant maneuver type for the `_1` vessel:
  - `course` → course change dominated
  - `speed` → speed change dominated

- **dominance_2**  
  Dominant maneuver type for the `_2` vessel.

---

## Contextual Information

- **region_type**  
  Dominant navigational region during the interaction (`harbor` or `open_sea`).

---

## Orientation Reference Columns (for transparency)

Because raw AIS data may contain symmetric duplicates or flipped ship ordering,
two reference columns are included to document the original orientation:

- **mmsi_1_example**  
  MMSI value appearing as `mmsi_1` in the first AIS record of the interaction.

- **mmsi_2_example**  
  MMSI value appearing as `mmsi_2` in the first AIS record of the interaction.

These columns allow unambiguous mapping between behavioral metrics (`_1` / `_2`)
and the physical vessels involved in each interaction.

---

## Interpretation Note

This dataset provides a **behavioral summary**, not full trajectories.
It captures how vessels respond during close-proximity encounters in a compact
and interpretable form, enabling statistical analysis and serving as a solid
foundation for subsequent visualization and behavior classification.
